In [ ]:
import logging
import numpy as np
import os

from tensorflow.keras.utils import Sequence

In [ ]:
# pip install import_ipynb nbformat

import import_ipynb
from Label_Utilities import build_label_dictionary
from Layer_Utilities import anchor_boxes, IoU, get_gt_data
from Image_Utilities import (apply_random_crop, apply_random_exposure_adjust,
                             apply_random_intensity_rescale,
                             apply_random_noise, apply_horizontal_flip,
                             read_image, resize_with_pad)

In [ ]:
"""The DataGenerator class inherits from the Sequence class of Keras to ensure
that it supports multi-processing. DataGenerator guarantees that the entire
dataset is used in one epoch.

The length of the entire epoch given a batch size is returned by the
__len__() method. Every request for a mini-batch of data is fulfilled by
the __getitem__() method. After every epoch, the on_epoch_end() method is
called to shuffle the entire batch if self.shuffle is True.
"""


class DataGenerator(Sequence):
    """Multi-threaded data generator.
    Each thread read a batch of images and their object labels
    
    # Arguments:
        args: User-defined configuration
        dictionary (dict): Dictionary of image filenames and object labels
        n_classes (int): Number of object classes
        feature_shapes (tensor): Shapes of ssd head feature maps
        n_anchors (int): Number of anchors per feature point (eg 4)
        shufle (bool): If dataset should be shuffled before sampling
    """
    def __init__(self,
                 args,
                 dictionary,
                 n_classes,
                 feature_shapes=[],
                 n_anchors=4,
                 horizontal_flip=0.,
                 random_crop = 0.,
                 random_exposure_adjust = 0,
                 random_intensity_rescale = 0.,
                 random_noise=0.,
                 shuffle=True):
        self.args = args
        self.dictionary = dictionary
        self.n_classes = n_classes
        self.keys = np.array(list(self.dictionary.keys()))
        self.input_shape = (args['height'], args['width'], args['channels'])
        self.feature_shapes = feature_shapes
        self.n_anchors = n_anchors
        
        # data augmentation
        self.horizontal_flip = horizontal_flip
        self.random_crop = random_crop
        self.random_exposure_adjust = random_exposure_adjust
        self.random_intensity_rescale = random_intensity_rescale
        self.random_noise = random_noise
        
        self.shuffle = shuffle

        self.on_epoch_end()
        self.get_n_boxes()

    def __len__(self):
        """Number of batches per epoch
        """
        batch_len = np.floor(len(self.dictionary) / self.args['batch_size'])
        return int(batch_len)

    def __getitem__(self, index):
        """Get a batch of data"""
        start_index = index * self.args['batch_size']
        end_index = start_index + self.args['batch_size']
        keys = self.keys[start_index:end_index]
        x, y = self.__data_generation(keys)
        return x, y

    def on_epoch_end(self):
        """Shuffle after each epoch"""
        if self.shuffle == True:
            np.random.shuffle(self.keys)

    def get_n_boxes(self):
        """Total number of bounding boxes"""
        self.n_boxes = 0
        for feature_shape in self.feature_shapes:
            self.n_boxes += np.prod(
                feature_shape) // 4  # // self.n_anchors ???
        logging.info(f"Data Generator: # boxes = {self.n_boxes}")
        return self.n_boxes

    def __data_generation(self, keys):
        """Generate train data: images and
        object detection ground truth labels
        
        # Arguments:
            keys (array): Randomly sampled keys (key is image filename)
        # Returns:
            x (tensor): Batch images
            y (tensor): Batch  classes, offsets and masks
        """
        # train input data
        x = np.zeros((self.args['batch_size'], *self.input_shape),
                     dtype=np.float32)
        # class ground truth
        dim = (self.args['batch_size'], self.n_boxes, self.n_classes)
        gt_class = np.zeros(dim)
        # offsets ground truth
        dim = (self.args['batch_size'], self.n_boxes, 4)
        gt_offset = np.zeros(dim)
        # masks of valid bounding boxes
        gt_mask = np.zeros(dim)

        for i, key in enumerate(keys):
            # images are assumed to be stores in self.args['data_path']
            # key is the image filename
            image_path = os.path.join(self.args['data_path'], key)
            # a label entry is made of 4-dim bounding box coords
            # and 1-dim class label
            labels = self.dictionary[key]

            # read image and labels
            image, labels = read_image(image_path, labels=labels)

            # horizontal_flip
            image, labels = apply_horizontal_flip(image, labels, self.horizontal_flip)
            
            # random_crop
            image, labels = apply_random_crop(image, labels, self.random_crop)
            
            # random_intensity_rescale
            image = apply_random_intensity_rescale(image, self.random_intensity_rescale)
            
            # random_exposure_adjust
            image = apply_random_exposure_adjust(image, self.random_exposure_adjust)
            
            # random noise
            image = apply_random_noise(image, self.random_noise)
            
            if image.shape[0:2] != self.input_shape[0:2]:
                image, labels = resize_with_pad(image,
                                                labels,
                                                *self.input_shape[0:2])
            assert (image.shape == self.input_shape)
            
            x[i] = image
            # 4 bounding box coords are 1st four items of labels
            # last item is object class label
            labels = np.array(labels)
            boxes = labels[:, 0:-1]

            for index, feature_shape in enumerate(self.feature_shapes):
                # generate anchor boxes
                anchors = anchor_boxes(
                    feature_shape,
                    image.shape,
                    index=index,
                    n_layers=self.args['layers'],
                    aspect_ratios=self.args['aspect_ratios'])
                # each feature layer has a row of anchor boxes
                anchors = np.reshape(anchors, [-1, 4])
                # compute IoU of each anchor box
                # with respect to each bounding boxes
                iou = IoU(anchors, boxes)
                # generate ground truth class, offset and mask
                gt = get_gt_data(iou,
                                 n_classes=self.n_classes,
                                 anchors=anchors,
                                 labels=labels,
                                 normalize=self.args['normalize'],
                                 threshold=self.args['threshold'])
                gt_cls, gt_off, gt_msk = gt

                if index == 0:
                    cls = np.array(gt_cls)
                    off = np.array(gt_off)
                    msk = np.array(gt_msk)
                else:
                    cls = np.append(cls, gt_cls, axis=0)
                    off = np.append(off, gt_off, axis=0)
                    msk = np.append(msk, gt_msk, axis=0)

            gt_class[i] = cls
            gt_offset[i] = off
            gt_mask[i] = msk
        y = [gt_class, np.concatenate((gt_offset, gt_mask), axis=-1)]
        return x, y

In [ ]:
if __name__ == "__main__":
    from matplotlib import pyplot as plt
    from matplotlib.patches import Rectangle
    
    # pip install import_ipynb nbformat

    import import_ipynb  # neccessary to import a Jupyter notebook
    from Label_Utilities import build_label_dictionary
    from Boxes import show_boxes

    args = {
        # params execution
        'dataset': 'voc2017',

        # configuration
        'aspect_ratios': (1., 2., 0.5, 3., 1./3.),
        'height': 300,
        'width': 300,
        'channels': 3,
        'layers': 4,
        'normalize': False,
        'batch_size': 1,
        'data_path': os.path.join('datasets', 'voc2007'),

        # params Non Maximum Suppresion
        'soft_nms': False,
        'iou_threshold': 0.3, # remove overlapping predictions with iou>threshold
        'class_threshold': 0.8, # if max obj probability is less than theshold (def 0.8) break

        # params: train
        'train_labels': 'labels_train.csv',
        'threshold': 0.6,
    }
    

    # build args to data generator
    path = os.path.join(args['data_path'], args['train_labels'])
    dictionary, classes = build_label_dictionary(path)
    n_anchors = len(args['aspect_ratios']) + 1
    feature_shapes = (
        (18, 18, 24), (9, 9, 24), (5, 5, 24), (3, 3, 24))  # el tercer campo entre 4 igual al numero de aspect+ratios +1
    
    assert(args['layers'] == len(feature_shapes))

    data_generator = DataGenerator(args,
                                   dictionary,
                                   n_classes=len(classes),
                                   feature_shapes=feature_shapes,
                                   n_anchors=n_anchors,
                                   random_crop = 1.0,
                                   random_exposure_adjust=0.,
                                   random_intensity_rescale=0.,
                                   horizontal_flip = 0.,
                                   random_noise = 0.,
                                   shuffle=False)

    i = 0
    for image_batch, (class_batch, offset_batch) in data_generator:
        j = np.random.randint(0, args['batch_size'])
        
        image, classes, offsets = image_batch[j], class_batch[j], offset_batch[j]

        class_names, rects, _, _ = show_boxes(args,
                                              image,
                                              classes,
                                              offsets,
                                              feature_shapes,
                                              show=True)

        print(class_names, offsets.shape)



        for index, feature_shape in enumerate(feature_shapes):
            anchor = anchor_boxes(feature_shape,
                                  image.shape,
                                  index=index,
                                  n_layers=args['layers'],
                                  aspect_ratios=args['aspect_ratios'])
            anchor = np.reshape(anchor, (-1, 4))
            if index == 0:
                anchors = anchor
            else:
                anchors = np.concatenate((anchors, anchor), axis=0)

        fig, ax = plt.subplots(1)
        ax.imshow(image)

        # get all non-zero (non-background) objects
        objects = np.argmax(classes, axis=1)
        # non-zero indexes are not background
        nonbg = np.nonzero(objects)[0]

        for idx in nonbg:
            # batch, row, col, box
            anchor = anchors[idx]
            # default anchor box format is
            # xmin, xmax, ymin, ymax
            w0 = anchor[1] - anchor[0]
            h0 = anchor[3] - anchor[2]
            x0 = anchor[0]
            y0 = anchor[2]
            offset = offsets[idx]


            box = anchor + offset[0:4]

            # default anchor box format is
            # xmin, xmax, ymin, ymax
            w1 = box[1] - box[0]
            h1 = box[3] - box[2]
            x1 = box[0]
            y1 = box[2]

            iou = IoU(np.expand_dims(anchors[idx], axis=0), np.expand_dims(box, axis=0))

            if iou > args['threshold']:

                print("IoU", iou)
                print("anchor:", anchor)
                print("offset: ", offset[0:4])
                print("boxes", box)
                rect = Rectangle((x0, y0),
                             w0,
                             h0,
                             linewidth=2,
                             edgecolor='blue',
                             facecolor='none')
                ax.add_patch(rect)




                print("\n")
                rect = Rectangle((x1, y1),
                             w1,
                             h1,
                             linewidth=2,
                             edgecolor='white',
                             facecolor='none')
                ax.add_patch(rect)
        plt.show()
        i = i + 1
        if i > 8:
            break